In [2]:
import os
from scipy import misc
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow

Using TensorFlow backend.


In [3]:
def create_dir(directory):
    os.makedirs(directory)

In [4]:
def check_dir(directory):
    if not os.path.exists(directory):
        create_dir(directory)

### Split and transform train data to train and val

In [15]:
width = height = 32
image_size = (width, height)
dir_transformed = '../output/transformed'+str(width)

In [7]:
%%time
dir_train_transformed = os.path.join(dir_transformed, 'train')
dir_val_transformed = os.path.join(dir_transformed, 'validation')

dir_train = '../input/train'
test_size = 0.25
    
for ind, folder in enumerate(os.listdir(dir_train)):
        subfolder = os.path.join(dir_train, folder)
        if os.path.isdir(subfolder):
            image_class = folder[:3]
            
            val_size = int(test_size * len(os.listdir(subfolder)))
            val_count = 0
            dir_train_to_save = os.path.join(dir_train_transformed, image_class)
            dir_val_to_save = os.path.join(dir_val_transformed, image_class)
            check_dir(dir_train_to_save)
            check_dir(dir_val_to_save)
            
            if ind % 10 == 0:
                print('folder - {}, validation size - {}, total size - {}'.format(folder, val_size, len(os.listdir(subfolder))))
            for image_file in os.listdir(subfolder):
                if not image_file.startswith('.'):
                    name = os.path.join(subfolder, image_file)
                    image = misc.imread(name)
                    if len(image.shape) == 2:
                        image = np.stack((image,)*3)
                    image = misc.imresize(image, image_size, interp='bicubic')
                    if val_count < val_size:
                        fold_to_save = os.path.join(dir_val_to_save, image_file)
                        misc.imsave(fold_to_save, image)
                        val_count = val_count + 1
                    else:
                        fold_to_save = os.path.join(dir_train_to_save, image_file)
                        misc.imsave(fold_to_save, image)

folder - 010.beer-mug, validation size - 16, total size - 64
folder - 020.brain-101, validation size - 13, total size - 53
folder - 030.canoe, validation size - 18, total size - 74
folder - 040.cockroach, validation size - 23, total size - 94
folder - 050.covered-wagon, validation size - 16, total size - 67
folder - 060.duck, validation size - 14, total size - 57
folder - 070.fire-extinguisher, validation size - 13, total size - 54
folder - 080.frog, validation size - 21, total size - 86
folder - 090.gorilla, validation size - 45, total size - 182
folder - 100.hawksbill-101, validation size - 15, total size - 63
folder - 110.hourglass, validation size - 13, total size - 55
folder - 120.joy-stick, validation size - 25, total size - 100
folder - 130.license-plate, validation size - 15, total size - 61
folder - 140.menorah-101, validation size - 14, total size - 59
folder - 150.octopus, validation size - 20, total size - 81
folder - 160.pez-dispenser, validation size - 13, total size - 53

### Transform test data

In [9]:
%%time
dir_test_transformed = os.path.join(dir_transformed, 'test/test')
dir_test = '../input/test'

check_dir(dir_test_transformed)
for ind, image_name in enumerate(os.listdir(dir_test)):
        if ind % 600 == 0:
            print('{}'.format(ind))
        if not image_name.startswith('.'):
            name = os.path.join(dir_test, image_name)
            image = misc.imread(name)
            if len(image.shape) == 2:
                image = np.stack((image,)*3)
            image = misc.imresize(image, image_size, interp='bicubic')
            fold_to_save = os.path.join(dir_test_transformed, image_name)
            misc.imsave(fold_to_save, image)

0
600
1200
1800
2400
3000
3600
4200
4800
5400
6000
6600
7200
CPU times: user 1min 25s, sys: 9.22 s, total: 1min 34s
Wall time: 1min 56s


In [ ]:
def save(dir_from, dir_to):
    for ind, folder in enumerate(os.listdir(dir_from)):
        subfolder = os.path.join(dir_from, folder)
        if os.path.isdir(subfolder):
            image_class = folder[:3]
            dir_train_to_save = os.path.join(dir_to, image_class)
            check_dir(dir_train_to_save)
            
            if ind % 10 == 0:
                print('folder - {}'.format(folder))
            for image_file in os.listdir(subfolder):
                if not image_file.startswith('.'):
                    name = os.path.join(subfolder, image_file)
                    image = misc.imread(name)
                    if len(image.shape) == 2:
                        image = np.stack((image,)*3)
                    image = misc.imresize(image, image_size, interp='bicubic')
                    fold_to_save = os.path.join(dir_train_to_save, image_file)
                    misc.imsave(fold_to_save, image)

In [ ]:
save('data/train', 'data/train_resized')

In [ ]:
dir_train = 'data/train_resized'
dir_train_augmented = 'data/train_augmented'
samples_threshhold = 775

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for ind, folder in enumerate(os.listdir(dir_train)):
    subfolder = os.path.join(dir_train, folder)
    if os.path.isdir(subfolder):            
        dir_train_to_save = os.path.join(dir_train_transformed, image_class)
        check_dir(dir_train_to_save)

        if ind % 20 == 0:
            print('folder - {}'.format(folder))
        needed_augmented_images = samples_threshhold - len(os.listdir(subfolder))
        dir_to_save = os.path.join(dir_train_augmented, folder)
        check_dir(dir_to_save)
        
        print(needed_augmented_images, ' ###### ', folder)
        for image_file in os.listdir(subfolder):
            if not image_file.startswith('.'):
                name = os.path.join(subfolder, image_file)
                image = misc.imread(name)
                #misc.imsave(dir_to_save, image)
                
                image = image.reshape((1,) + image.shape)
                
                for batch in datagen.flow(image, batch_size=1,
                      save_to_dir=dir_to_save, save_format='jpeg'):
                    needed_augmented_images -= 1
                    if needed_augmented_images < 0:
                        break

### Transform images for keras

In [ ]:
train_df = pd.read_csv('data/data_analys.csv', index_col = 0)

In [ ]:
images = np.zeros((train_df.shape[0], 64, 64, 3), dtype='uint8')
targets = np.zeros((train_df.shape[0],), dtype='object')

In [ ]:
dir_train = 'data/train_transformed'
img_size=(64, 64)

i =0
for ind, image_name in enumerate(os.listdir(dir_train)):
    image_file = os.path.join(dir_train, image_name)
    if ind % 2000 == 0:
        print('{}'.format(ind))
    if not image_file.startswith('.'):
        #name = os.path.join(subfolder, image_file)
        image = misc.imread(image_file)
        if len(image.shape) == 2:
            image = np.stack((image,)*3)
        images[i] = misc.imresize(image, img_size, interp='bicubic')
        targets[i] = int(image_name[:3])
        i = i+1

In [ ]:
np.save('data/keras/train_images.npy', images)
np.save('data/keras/targets.npy', targets)

In [ ]:
%%time
dir_test_name = 'data/test_transformed'
dir_test = os.listdir(dir_test_name)
img_size=(64, 64)

test_images = np.zeros((len(dir_test), img_size[0], img_size[1], 3), dtype='uint8')

i = 0
for ind, image_name in enumerate(os.listdir(dir_test_name)):
        if ind % 600 == 0:
            print('{}'.format(ind))
        if not image_name.startswith('.'):
            name = os.path.join(dir_test_name, image_name)
            image = misc.imread(name)
            if len(image.shape) == 2:
                image = np.stack((image,)*3)
            test_images[i] = misc.imresize(image, img_size, interp='bicubic')
            i = i+1

In [ ]:
np.save('data/keras/test_images.npy', test_images)